In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.4/715.4 kB 7.1 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import cv2 as cv
import numpy as np
from ultralytics import YOLO

# Load model
m = YOLO("best.pt")

# Predict with streaming to avoid RAM accumulation
results = m.predict("/content/30440.jpg", stream=True)

# Iterate over results as they are generated
for r in results:
    img = np.copy(r.orig_img)
    img_name = Path(r.path).stem

    # Iterate over each object contour
    for ci, c in enumerate(r):
        label = c.names[c.boxes.cls.tolist().pop()]
        class_id = c.boxes.cls.tolist().pop()  # Get class ID
        b_mask = np.zeros(img.shape[:2], dtype=np.uint8) # 검은색 mask 생성

        # Create contour mask
        contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)# 객체의 윤곽선 좌표 가져오기
        cv.drawContours(b_mask, [contour], -1, 255, cv.FILLED)# 검은색 배경에 객체의 윤곽선 좌표 가져오기

        # Extract object using the mask
        masked_object = cv.bitwise_and(img, img, mask=b_mask)

        # Find the bounding box of the masked object to crop tightly
        x, y, w, h = cv.boundingRect(b_mask)
        cropped_object = masked_object[y:y+h, x:x+w]

        # Create a new black image of the same size as the original
        height, width = img.shape[:2]
        centered_img = np.zeros((height, width, 3), dtype=np.uint8)

        # Calculate the position to place the object at the center
        obj_height, obj_width = cropped_object.shape[:2]
        x_offset = (width - obj_width) // 2
        y_offset = (height - obj_height) // 2

        # Place the object in the center of the black image
        centered_img[y_offset:y_offset+obj_height, x_offset:x_offset+obj_width]= cropped_object

        # Generate a unique file name for each detected object
        save_path = f"/content/{img_name}_{label}_{class_id}_{ci}.jpg"
        cv.imwrite(save_path, centered_img)


image 1/1 /content/30440.jpg: 256x416 1 Anger, 263.3ms
Speed: 16.6ms preprocess, 263.3ms inference, 35.4ms postprocess per image at shape (1, 3, 256, 416)


In [ ]:
import cv2
from pathlib import Path

def crop_to_mask(image_path, output_path):
    # 이미지 로드
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # 검은색 배경을 기준으로 마스크 생성
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # 마스크에서 경계 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"No contours found in image: {image_path}")
        return

    # 가장 큰 경계에 해당하는 영역을 찾아 크롭
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    cropped = image[y : y + h, x : x + w]

    # 크롭된 이미지 저장 (output_path에 파일 이름 추가)
    output_filename = Path(output_path) / f"{image_path.stem}_cropped{image_path.suffix}"
    cv2.imwrite(str(output_filename), cropped)

# 예제 사용
image_path = Path("/content/30440_Anger_0.0_0.jpg")
output_path = "/content/"
crop_to_mask(image_path, output_path)

In [ ]:
import cv2
from pathlib import Path


def crop_to_mask(image_path, output_path):
    # 이미지 로드
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # 검은색 배경을 기준으로 마스크 생성
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # 마스크에서 경계 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"No contours found in image: {image_path}")
        return

    # 가장 큰 경계에 해당하는 영역을 찾아 크롭
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    cropped = image[y : y + h, x : x + w]

    # 크롭된 이미지 저장
    cv2.imwrite(str("/content/"), cropped)


# 입력 및 출력 디렉토리 설정
input_dir = Path(
    "/content/30440_Anger_0.0_0.jpg"
)
output_dir = Path(
    "/content/"
)

# 입력 디렉토리의 모든 이미지에 대해 반복
for category in ["test", "train", "valid"]:
    for emotion in ["Angry", "Happy", "Neutral", "Sad", "Surprised"]:
        current_dir = input_dir / category / emotion
        output_current_dir = output_dir / category / emotion
        output_current_dir.mkdir(parents=True, exist_ok=True)  # 출력 디렉토리 생성

        for image_path in current_dir.glob("*.jpg"):
            output_image_path = output_current_dir / image_path.name
            crop_to_mask(image_path, output_image_path)